In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split


IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def collect_image_paths(root_dir):
    exts = (".jpg", ".jpeg", ".png")
    return sorted(p for p in Path(root_dir).rglob("*") if p.suffix.lower() in exts)

def build_label_df(paths):
    records = []
    cat_set = {
        "Abyssinian","Birman","Bengal","Bombay","British",
        "Egyptian","Maine","Persian","american","english"
    }
    for p in paths:
        label_str = p.stem.split("_")[0]
        binary = 0 if label_str in cat_set else 1
        records.append({"path": str(p), "label": binary})
    return pd.DataFrame.from_records(records)

def df_to_dataset(df, shuffle=True):
    paths = df["path"].values
    labels = df["label"].values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(len(df))
    ds = ds.map(_load_and_preprocess, num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

def _load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img, tf.cast(label, tf.int32)


def build_resnet50_feature_extractor():
    base = ResNet50(weights="imagenet", include_top=False,
                    input_shape=(*IMG_SIZE,3))
    base.trainable = False
    inputs = layers.Input((*IMG_SIZE,3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inputs, outputs, name="resnet50_feature")

def build_resnet50_fine_tuner(fine_tune_at=140):
    base = ResNet50(weights="imagenet", include_top=False,
                    input_shape=(*IMG_SIZE,3))
    base.trainable = False
    for layer in base.layers[fine_tune_at:]:
        layer.trainable = True
    inputs = layers.Input((*IMG_SIZE,3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inputs, outputs, name="resnet50_finetune")


def compile_and_train(model, train_ds, val_ds, output_path,
                      epochs=10, lr=1e-4):
    checkpoint = ModelCheckpoint(
        output_path,
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[checkpoint]
    )
    return history

if __name__ == "__main__":
    root_folder = r"C:\Users\hdgn5\OneDrive\Masaüstü\Bilgisayar Proglamlama\images"
    paths = collect_image_paths(root_folder)
    df = build_label_df(paths)
    train_df, val_df = train_test_split(
        df, test_size=0.2,
        stratify=df["label"],
        random_state=42
    )
    train_ds = df_to_dataset(train_df, shuffle=True)
    val_ds   = df_to_dataset(val_df, shuffle=False)

    # 4.1 Feature Extraction
    feat_model = build_resnet50_feature_extractor()
    hist_feat = compile_and_train(
        feat_model,
        train_ds,
        val_ds,
        output_path="best_resnet50_feature.h5",
        epochs=5,
        lr=1e-4
    )

    feat_model.save("best_resnet50_feature.h5")

    # 4.2 Fine-Tuning
    ft_model = build_resnet50_fine_tuner(fine_tune_at=140)
    hist_ft = compile_and_train(
        ft_model,
        train_ds,
        val_ds,
        output_path="best_resnet50_finetune.h5",
        epochs=5,
        lr=1e-5
    )
    ft_model.save("best_resnet50_finetune.h5")
